## Experiment with finding the bit start times, using traces previously collected by otbn_traces.ipynb:

In [ ]:
import numpy as np
waves = [np.load('waves_p384_100M_half1half0_again.npy')]
#waves = [np.load('waves_p384_100M_half1half0.npy')]
#waves = [np.load('waves_p384_100M_64b_patterned_again.npy')]
#waves = [np.load('waves_p384_100M_half0half1.npy')]
#waves = [np.load('waves_p384_100M_160ones96zeros.npy')]
#waves = [np.load('waves_p384_100M_128ones128zeros128ones.npy')]
#waves = np.load('waves_p384_100M_130k_samples.npy')
#waves = np.load('waves_p384_100M_leading1.npy')

In [ ]:
samples = 800000
#samples = len(waves[0])
base = 0

In [ ]:
import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade
hv.extension('bokeh')
datashade(hv.Curve(waves[0][base:base+samples])).opts(width=2000, height=800)

### HPF helps clean up the trace, especially at higher samples rates:

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq 
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=9):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

filtered_waves = []
for wave in waves:
        filtered_waves.append(butter_highpass_filter(wave, 6e6, 100e6))

In [ ]:
#samples = len(waves[0])
samples = 600000
base = 0

#samples = 100
#base = 5000007
import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, shade, dynspread
hv.extension('bokeh')

w0 = datashade(hv.Curve(waves[0][base:base+samples]), cmap=['green'])
wf = datashade(hv.Curve((filtered_waves[0][base:base+samples])), cmap=['black'])

#(w0 * wf).opts(width=2000, height=900)
(wf).opts(width=2000, height=600)

### Take a moving average to more easily identify the idle periods which separate the bits of k:

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

In [ ]:
#mfw = moving_average(np.abs(filtered_waves[0]), 2000)
mfw = moving_average(np.abs(filtered_waves[0]), 1000)

#samples = 600000
samples = len(mfw)
base = 0

mwf = datashade(hv.Curve(mfw[base:base+samples]), cmap=['black'])
mwf.opts(width=2000, height=600)

In [ ]:
from scipy.signal import find_peaks
peaks, _ = find_peaks(-mfw[base:base+samples], distance=20000)

In [ ]:
len(peaks)#, peaks

### Extract the bit start times: skip the first three peaks, then take every other:

In [ ]:
raw_bit_starts = peaks[3::2]

In [ ]:
len(raw_bit_starts)

In [ ]:
# trim the end:
raw_bit_starts = raw_bit_starts[:256]

In [ ]:
raw_bit_starts[0], raw_bit_starts[-1]

In [ ]:
# visualize the start times:
from operator import mul
from functools import reduce

starts = [hv.VLine(b) for b in raw_bit_starts]
(mwf * reduce(mul, starts)).opts(width=2000, height=600)

In [ ]:
deltas = []
for i in range(len(raw_bit_starts)-2):
    delta = raw_bit_starts[i+1] - raw_bit_starts[i]
    deltas.append(delta)
    #print(delta)
    

In [ ]:
duration = int(np.average(deltas))
duration, max(deltas)-min(deltas)

In [ ]:
duration*384

### Perhaps it's better to take the average bit duration as the duration of each bit?

(spoiler: no it's not)

In [ ]:
even_starts = []
for i in range(256):
    even_starts.append(raw_bit_starts[0] + i*duration)
    #even_starts.append(raw_bit_starts[0] + i*54805)

In [ ]:
# which to use:
bit_starts = raw_bit_starts
#bit_starts = even_starts

# Superimpose all the bits!
Plot overlayed bit traces to visualize alignment and guess at success of time extraction:

In [ ]:
bits = []
bit_size = duration
for start in bit_starts:
    bits.append(filtered_waves[0][start:start+bit_size])

In [ ]:
# Can't plot more than 22 bits due to palette:
#numbits = len(bits)
numbits = 22

import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, shade, dynspread
from holoviews import opts
import datashader as ds

hv.extension('bokeh')

lines = {i: hv.Curve(np.asarray(bits[i])) for i in range(len(bits[:numbits]))}
linespread = dynspread(datashade(hv.NdOverlay(lines, kdims='k'), aggregator=ds.by('k', ds.count())))
linespread.opts(opts.RGB(width=2400, height=1200))


In [ ]:
# Can plot all the bits, but it's slow:
#numbits = len(bits)
numbits = 10

import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, shade, dynspread
hv.extension('bokeh')

#xrange = range(duration)

from operator import mul
from functools import reduce

curves = [hv.Curve(bits[i]) for i in range(numbits)]

reduce(mul, curves).opts(width=2000, height=900)

# Average 'one' and 'zero':
Let's see if simple difference-of-means reveals anything.

In [ ]:
avg_trace = np.zeros(duration)
avg_ones = np.zeros(duration)
avg_zeros = np.zeros(duration)

for i in range(len(bit_starts)):
    avg_trace += bits[i][:duration]
    # CAUTION: this is for k = {128 ones, 128 zeros}; adjust for different k:
    if i < 128:
        avg_ones += bits[i][:duration]
    else:
        avg_zeros += bits[i][:duration]

avg_trace /= len(bit_starts)
avg_ones /= len(bit_starts)/2
avg_zeros /= len(bit_starts)/2


In [ ]:
import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, shade, dynspread
hv.extension('bokeh')

xrange = range(duration)

cavg_all = datashade(hv.Curve(avg_trace), cmap=['black'])
cavg_ones = datashade(hv.Curve(avg_ones), cmap=['blue'])
cavg_zeros = datashade(hv.Curve(avg_zeros), cmap=['green'])

cdiff = datashade(hv.Curve((avg_ones - avg_zeros)), cmap=['red'])

#(cavg_all * cavg_ones * cavg_zeros).opts(width=2000, height=900)
#(cavg_zeros * cavg_ones).opts(width=2000, height=900)
#(cavg_ones * cavg_zeros).opts(width=2000, height=900)
(cdiff).opts(width=2000, height=600)

In [ ]:
markers = np.where(abs(avg_ones - avg_zeros) > 0.014)[0] # half/half

In [ ]:
len(markers)#, markers

In [ ]:
# save markers for attacking other traces:
#np.save('markers_p384_100M.npy', markers)
#markers = np.load('markers_p384_100M.npy')

In [ ]:
scores = []
#for b in bit_starts:
for b in range(len(bit_starts)):
    score = 0
    for marker in markers:
        #score += abs(filtered_waves[0][b + marker])
        score += abs(bits[b][marker])
    scores.append(score)

In [ ]:
cscores = hv.Curve(scores)
(cscores * hv.VLine(128)).opts(width=2000, height=600)

In [ ]:
np.average(scores[12:128]), np.average(scores[128:256])